## Extra Tree


In [12]:
import matplotlib.pyplot as plt
import xgboost as xgb
import mltools as ml
import numpy as np
from sklearn import *

In [2]:
X = np.genfromtxt('data/X_train.txt',delimiter = ',')
Y = np.genfromtxt('data/Y_train.txt',delimiter = ',')
X,Y = ml.shuffleData(X,Y)
Xtr,Xva,Ytr,Yva = ml.splitData(X,Y,0.75)
Xte = np.genfromtxt('data/X_test.txt',delimiter = ',')

In [3]:
extra_tree_classifier = ensemble.ExtraTreesClassifier(
    n_estimators=200, max_depth=200, min_samples_split=6, min_samples_leaf=3, max_features='auto')

In [4]:
extra_tree_classifier.fit(Xtr, Ytr)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=200, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=3, min_samples_split=6,
                     min_weight_fraction_leaf=0.0, n_estimators=200,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [5]:
extra_tree_classifier_roc = metrics.roc_auc_score(
    Yva, extra_tree_classifier.predict_proba(Xva)[:,1])
print("validation roc:", extra_tree_classifier_roc)

validation roc: 0.6892280707867975


In [10]:
random_forest_classifier = ensemble.RandomForestClassifier(
    n_estimators=1200, min_samples_leaf=5, n_jobs=-1, oob_score=True)

print("training started")
random_forest_classifier.fit(Xtr, Ytr)
random_forest_classifier_roc = metrics.roc_auc_score(
    Yva, random_forest_classifier.predict_proba(Xva)[:,1])
print(random_forest_classifier_roc)

print("training error:", 1 - random_forest_classifier.score(Xtr, Ytr))
print("validation error:", 1 - random_forest_classifier.score(Xva, Yva))

training started
0.7266437835043762
training error: 0.017424106340937717
validation error: 0.34536637931034486


In [13]:
bat =  xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=5, min_child_weight=3, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.8, reg_lambda=0.5, scale_pos_weight=1, seed=4396,
             silent=None, subsample=0.8,verbosity=1)
bat.fit(Xtr,Ytr)

metrics.roc_auc_score(Yva,bat.predict(Xva))

[00:29:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.7251058491453661

In [36]:
ada_classifier = ensemble.AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=5, max_features=15),
               n_estimators=1100, learning_rate=0.10)

ada_classifier.fit(Xtr, Ytr)

ada_classifier_roc = metrics.roc_auc_score(
   Ytr, ada_classifier.predict_proba(Xtr)[:,1])
print(ada_classifier_roc)

1.0


In [33]:
classifier_list = [
    ("random_forest", random_forest_classifier, 18),
    ("extra_tree_classifier", extra_tree_classifier, 6),
    ("gradient_boosting", bat, 20),
     ("ada_boosting", ada_classifier, 3),
]


y_validation_hat_list = []
for classifier in classifier_list:
    pyva = classifier[1].predict(Xte)
    for i in range(classifier[2]):
        y_validation_hat_list.append(pyva)
        
y_validation_hat_average = np.mean(np.array(y_validation_hat_list), axis=0)

#voting_roc = metrics.roc_auc_score(Yva, y_validation_hat_average)

In [34]:
Xte = np.genfromtxt('data/X_test.txt', delimiter=',')
Yte = np.vstack((np.arange(Xte.shape[0]),y_validation_hat_average)).T
np.savetxt('Y_submit.txt',Yte,'%d, %.2f',header='Id,Predicted',comments='',delimiter=',')

# 